In [3]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [29]:
df = pd.read_csv("../data/adult.csv")
df.head(10)

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25
5,5,37,Private,Masters,Married-civ-spouse,Exec-managerial,Wife,White,Female,40,United-States,<=50K,44
6,6,49,Private,9th,Married-spouse-absent,Other-service,Not-in-family,Black,Female,16,Jamaica,<=50K,49
7,7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,307
8,8,31,Private,Masters,Never-married,Prof-specialty,Not-in-family,White,Female,50,United-States,>50K,330
9,9,42,Private,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,40,United-States,>50K,303


# Task 1
Print the count of men and women in the dataset.

In [12]:
men_in_dataset = df[df["sex"] == "Male"]
women_in_dataset = df[df["sex"] == "Female"]
print(men_in_dataset.shape[0])
print(women_in_dataset.shape[0])

21790
10771


# Task 2
Find the average age of men in dataset

In [15]:
average_age_of_men = men_in_dataset["age"].mean()
print(average_age_of_men)

39.43354749885268


# Task 3
Get the percentage of people from Poland (native-country)

In [16]:
percentage_of_people_from_poland = (df[df["native-country"] == "Poland"].shape[0] * 100) / len(df)
print(percentage_of_people_from_poland)

0.18426952489174164


# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [21]:
mean_of_the_age_for_people_earn_gt_50k = df[df["salary"] == ">50K"]["age"].mean()
mean_of_the_age_for_people_earn_lte_50k = df[df["salary"] == "<=50K"]["age"].mean()
standard_deviation_of_the_age_for_people_earn_gt_50k = df[df["salary"] == ">50K"]["age"].std()
standard_deviation_of_the_age_for_people_earn_lte_50k = df[df["salary"] == "<=50K"]["age"].std()
print(mean_of_the_age_for_people_earn_gt_50k)
print(mean_of_the_age_for_people_earn_lte_50k)
print(standard_deviation_of_the_age_for_people_earn_gt_50k)
print(standard_deviation_of_the_age_for_people_earn_lte_50k)

44.24984058155847
36.78373786407767
10.519027719851826
14.02008849082488


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [25]:
people_without_higher_education = df[
    (df["salary"] == ">50K") &
    ~(df["education"].isin(["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]))
]
print(people_without_higher_education.shape[0])

3306


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [28]:
statistic_base_on_education = df.groupby("education")["age"].describe()
statistic_base_on_education

,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,933.0,37.429796,16.720713,17.0,22.00,34.0,52.0,90.0
11th,1175.0,32.355745,15.545485,17.0,18.00,28.0,43.0,90.0
12th,433.0,32.000000,14.334625,17.0,19.00,28.0,41.0,79.0
1st-4th,168.0,46.142857,15.615625,19.0,33.00,46.0,57.0,90.0
5th-6th,333.0,42.885886,15.557285,17.0,29.00,42.0,54.0,84.0
7th-8th,646.0,48.445820,16.092350,17.0,34.25,50.0,61.0,90.0
9th,514.0,41.060311,15.946862,17.0,28.00,39.0,54.0,90.0
Assoc-acdm,1067.0,37.381443,11.095177,19.0,29.00,36.0,44.0,90.0
Assoc-voc,1382.0,38.553546,11.631300,19.0,30.00,37.0,46.0,84.0


# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [36]:
maried_men_earn_gt_50k = df[(df["marital-status"].str.startswith("Married")) & (df["salary"] == ">50k")].shape[0]
non_married_men_earn_gt_50k = df[~(df["marital-status"].str.startswith("Married")) & (df["salary"] == ">50K")].shape[0]
maried_men_earn_lte_50k = df[(df["marital-status"].str.startswith("Married")) & (df["salary"] == "<=50k")].shape[0]
non_married_men_earn_lte_50k = df[~(df["marital-status"].str.startswith("Married")) & (df["salary"] == "<=50K")].shape[0]
print(f"Married men, who earns more than 50k are earning more than non-married: {maried_men_earn_gt_50k > non_married_men_earn_gt_50k}")
print(f"Married men who earns less than 50k are earning more than non-married: {maried_men_earn_lte_50k > non_married_men_earn_lte_50k}")

Married men, who earns more than 50k are earning more than non-married: False
Married men who earns less than 50k are earning more than non-married: False


# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [41]:
max_hours_per_week = df["hours-per-week"].max()
how_many_persons_works_maximum_hours = df[df["hours-per-week"] == max_hours_per_week].shape[0]
how_many_persons_works_maximum_hours

85

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [47]:
numeric_columns = df.select_dtypes(include=[np.number]).drop(columns=["Unnamed: 0"])
correlation_matrix = numeric_columns.corr()
print(correlation_matrix)
print()
print("Age and Salary: There is a positive correlation between age and salary (0.201774).")
print("This suggests that, on average, as a person's age increases, their salary tends to increase as well.")
print("This connection could be related to career progression and experience.")
print("--------------------------------------------------")
print("Hours per Week and Salary:")
print("There is a positive correlation between hours per week worked and salary (0.196916).")
print("This indicates that individuals who work more hours per week tend to have higher salaries on average.")
print("This connection might reflect higher earnings for individuals with more working hours.")
print("--------------------------------------------------")
print("Age and Hours per Week:")
print("There is a relatively weak positive correlation between age and hours per week worked (0.068756).")
print("This connection suggests that, on average, older individuals tend to work slightly more hours per week, but the correlation is not very strong.")
print("--------------------------------------------------")

                     age  hours-per-week  salary K$
age             1.000000        0.068756   0.201774
hours-per-week  0.068756        1.000000   0.196916
salary K$       0.201774        0.196916   1.000000

Age and Salary: There is a positive correlation between age and salary (0.201774).
This suggests that, on average, as a person's age increases, their salary tends to increase as well.
This connection could be related to career progression and experience.
--------------------------------------------------
Hours per Week and Salary:
There is a positive correlation between hours per week worked and salary (0.196916).
This indicates that individuals who work more hours per week tend to have higher salaries on average.
This connection might reflect higher earnings for individuals with more working hours.
--------------------------------------------------
Age and Hours per Week:
There is a relatively weak positive correlation between age and hours per week worked (0.068756).
This connec